# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберите победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [148]:
!pip install razdel
from razdel import tokenize
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [88]:
from google.colab import files
uploaded = files.upload()

Saving labeled.csv to labeled (2).csv


In [89]:
data = pd.read_csv('labeled.csv')
data.toxic.value_counts(normalize=True) #66 процентов твитов нетоксичные

0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64

In [92]:
#делим данные на обучающую и тестовую выборки
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [93]:
#векторизация с дефолтной токенизацией
default_vectorizer = CountVectorizer()
X1 = default_vectorizer.fit_transform(train.comment)
X1_test = default_vectorizer.transform(test.comment)
y1 = train.toxic.values
y1_test = test.toxic.values

In [94]:
#обучение классификатора при дефолтной токенизации
clf1 = MultinomialNB(alpha=1.)
clf1.fit(X1, y1)

preds1 = clf1.predict(X1_test)
print(classification_report(y1_test, preds1, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90       958
         1.0       0.89      0.65      0.75       484

    accuracy                           0.85      1442
   macro avg       0.87      0.80      0.82      1442
weighted avg       0.86      0.85      0.85      1442



In [95]:
#векторизация с токенизацией razdel_tokenize
def natashatokens (data):
    tokenized = list(tokenize(data))
    tokens = list ()
    for t in tokenized:
      tokens.append (t.text)
    return tokens

In [96]:
natasha_vectorizer = CountVectorizer(tokenizer = natashatokens)
X2 = natasha_vectorizer.fit_transform(train.comment)
X2_test = natasha_vectorizer.transform(test.comment)
y2 = train.toxic.values
y2_test = test.toxic.values

In [97]:
#обучение классификатора при токенизации razdel_tokenize
clf2 = MultinomialNB(alpha=1.)
clf2.fit(X2, y2)

natashapreds = clf2.predict(X2_test)
print(classification_report(y2_test, natashapreds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.83      0.96      0.89       958
         1.0       0.89      0.61      0.72       484

    accuracy                           0.84      1442
   macro avg       0.86      0.79      0.81      1442
weighted avg       0.85      0.84      0.84      1442



In [ ]:
#если сравнить метрики, то мы увидим, что оба классификатора работают приблизительно одинаково
#но в метриках recall и f1-score классификатор с кастомной векторизацией работает стально немного хуже

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [64]:
#таблица с абсолютными частотностями
df = pd.DataFrame({"я" : [1, 1, 3, 1, 0],
                 "ты" : [1, 1, 0, 0, 0],
                 "и" : [1, 1, 1, 0, 0],
                 "только" : [0, 0, 1, 1, 0],
                 "не" : [0, 0, 0, 1, 0],
                 "он" : [0, 0, 0, 0, 1]})
df = df.rename({0: "я и ты", 1: "ты и я", 2: "я, я и только я", 3: "только не я", 4: "он"})
df

,я,ты,и,только,не,он
я и ты,1,1,1,0,0,0
ты и я,1,1,1,0,0,0
"я, я и только я",3,0,1,1,0,0
только не я,1,0,0,1,1,0
он,0,0,0,0,0,1


In [65]:
#total number of docs N
df.index.size

5

In [66]:
#таблица с tfidf значениями
values = []
for i in df.index:
  #print (i)
  tfidf_list = []
  for column in df.columns:
    #print (column)
    x = df[column].loc[i]
    words_count = 0
    for j in df.loc [i]:
      words_count += j
    tf = x / words_count
    tfidf = tf * math.log (df.index.size / np.count_nonzero(df[column]))
    tfidf_list.append (tfidf)
  values.append (tfidf_list)
values

[[0.07438118377140325, 0.3054302439580517, 0.1702752079219969, 0.0, 0.0, 0.0],
 [0.07438118377140325, 0.3054302439580517, 0.1702752079219969, 0.0, 0.0, 0.0],
 [0.13388613078852585,
  0.0,
  0.10216512475319815,
  0.18325814637483104,
  0.0,
  0.0],
 [0.07438118377140325, 0.0, 0.0, 0.3054302439580517, 0.5364793041447, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.6094379124341003]]

In [67]:
pd = pd.DataFrame(values, index = df.index, columns = df.columns)
pd

,я,ты,и,только,не,он
я и ты,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
ты и я,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
"я, я и только я",0.133886,0.00000,0.102165,0.183258,0.000000,0.000000
только не я,0.074381,0.00000,0.000000,0.305430,0.536479,0.000000
он,0.000000,0.00000,0.000000,0.000000,0.000000,1.609438


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [68]:
#Naive Bayes + CountVectorizer

vectorizer = CountVectorizer(lowercase=True, tokenizer = natashatokens, max_df=0.5, min_df=10, max_features=3000)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

clf = MultinomialNB(alpha=1., fit_prior=True)
clf.fit(X, y)
nb_preds = clf.predict(X_test)
nb_probas = clf.predict_proba(X_test)

print(classification_report(y_test, nb_preds))

              precision    recall  f1-score   support

         0.0       0.86      0.90      0.88       967
         1.0       0.78      0.69      0.73       475

    accuracy                           0.83      1442
   macro avg       0.82      0.80      0.81      1442
weighted avg       0.83      0.83      0.83      1442



In [73]:
#функция для нахождения 10 токсичных комментов
def find_toxic(probas):
  probas = [p[1] for p in probas]
  test['probas'] = probas

  res = test.sort_values(by = 'probas', ascending = False)[:10]
  res = res.reset_index(drop=True)

  for i in range(10):
    print('toxic:', res.loc[i].toxic)
    print('comment: ', res.loc[i].comment)
  return (res)

In [74]:
#топ-10 токсичных комментариев для классификатора Naive Bayes + CountVectorizer
print (find_toxic(nb_probas))

toxic: 0.0
comment:  Конечно, это резонансное событие требует отдельного треда для обсуждения и 19 тредов до этого шло обсуждение. Тред назывался нейтрально Рассстрел мусульман в новозеландской мечети . Но сегодня ОП, по совместительству борец за Арийскую Раssy и мамкин фашик назвал его в честь стрелка KEBAB REMOVER(англ. уничтожитель чурок ) и с утра пораньше решил в ОП-посте(заголовке треда) обозначить тред, как героизирующий(в первом треде) и оправдывающий(в третьем треде) терракт этого стрелка. Пик стронгли релейтед. 20 тред. В третьей строке заголовка ОП восхваляет и героизирует террориста, цитирую заголовок: KEBAB REMOVER тред 20 Продолжаем пиздеть тут. Слава Герою! Я репортил тред в форме репорта и в прикрепе в d , реакции никакой. Видимо, школомодеры сами симпатизируют нацизму и закрывают на это глаза. 21 тред. После того, как я пригрозил ему репортом, в этот раз мамкин фашик, убирает строку про героя и зачем-то вместо неё ставит нацистское приветствие дивизии СС Галичина. KEBA

In [71]:
#LogisticRegression + TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase=True, tokenizer = natashatokens, max_df=0.5, min_df=10, max_features=3000)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X, y)
lr_preds = clf.predict(X_test)
lr_probas = clf.predict_proba(X_test)

print(classification_report(y_test, lr_preds))

              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87       967
         1.0       0.73      0.76      0.74       475

    accuracy                           0.83      1442
   macro avg       0.80      0.81      0.81      1442
weighted avg       0.83      0.83      0.83      1442



In [72]:
#топ-10 токсичных комментариев для классификатора LogisticRegression + TfidfVectorizer
print (find_toxic(lr_probas))

toxic: 0.0
comment:  Ты крутая. Бэнкси отдыхает

toxic: 1.0
comment:  ебать ты конченый, нахуя ты это постишь?

toxic: 1.0
comment:  Валентин ты опять таблетки не выпил

toxic: 1.0
comment:  Да мы поняли что ты пидорьё ебаное, поляк наверное. Иди уроки выучи, портфель собери, че ты.

toxic: 1.0
comment:  Что. ты. несёшь. дегенерат?
toxic: 1.0
comment:  Вольнова же повесили на параше, ты про что?

toxic: 1.0
comment:  Я тебе дохуя пруфов привёл, а ты ебанный дебил не верил. Сося должен был вам сказать, но сося долбаёб и тебя за человека не держит, почему он тебе должен рассказывать о своих фейл, если его заявляения раньше звучали гордо . Ебать ты дибил.

toxic: 0.0
comment:  А ты вспомни чем Женя Лукашин салатницу мыл

toxic: 1.0
comment:  А почему ты упустил вот это Ты не упустил. Молодец.

toxic: 1.0
comment:  да пошел ты нахуй сашенька ебаный

   index                                            comment  toxic    probas
0    837                       Ты крутая. Бэнкси отдыхает\n    0.

In [ ]:
#в итоге ни один текст не совпал и не все тексты токсичные:
#в выборку классификатора Naive Bayes с CountVectorizer попали 3 нетоксичных комментария, LogisticRegression + TfidfVectorizer сработал чуть точнее
#практически во всех твитах есть маты и жаргон, что влияет на высокий уровень токсичности

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [142]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
russian_stopwords = set(stopwords.words('russian'))

train, test = train_test_split(data, test_size=0.1, shuffle=True)
train = train.reset_index(drop=True) 
test = test.reset_index(drop=True) 

vectorizer = CountVectorizer(stop_words = russian_stopwords, min_df = 10, max_df = 0.5)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [143]:
#Logistic Regression
clf = LogisticRegression(C = 0.2, solver = 'liblinear', class_weight = 'balanced')
clf.fit(X,y)

importance = clf.coef_

toxic = {}
for key, value in vectorizer.vocabulary_.items():
  toxic[value] = key

top = importance.argsort()
for i in top[0][:-6:-1]:
    print(toxic[i])

хохлы
хохлов
дебил
тебе
тупые


In [151]:
#Naive Bayes
clf = MultinomialNB(alpha=1.)
clf.fit(X,y)

importance = clf.coef_

toxic = {}
for key, value in vectorizer.vocabulary_.items():
  toxic[value] = key

top = importance.argsort()
for i in top[0][:-6:-1]:
    print(toxic[i])

это
просто
тебе
почему
всё


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


In [121]:
#для начинающих очень тяжелое задание
from IPython.display import Image
Image(url="https://i.pinimg.com/originals/6b/84/c6/6b84c674c24beae23ca5df2ffee1fd96.jpg",
     width=380, height=500)